# Extraindo informações sobre as turmas
Esse notebook tem como objetivo utilizar os datasets de matriculas e turmas para pegar gerar informações a respeito das turmas.

In [2]:
import pandas as pd
import numpy as np

In [3]:
matriculas = pd.read_csv("data/matriculas-de-2016.1.csv", sep=";")

In [4]:
# Pegando apenas as turmas de CeT
matriculas_cet = matriculas[matriculas.id_curso == 10320810]

In [5]:
matriculas_cet.descricao.unique()

array(['REPROVADO', 'APROVADO POR NOTA', 'INDEFERIDO', 'EXCLUIDA',
       'TRANCADO', 'CANCELADO', 'DESISTENCIA',
       'REPROVADO POR MÉDIA E POR FALTAS', 'APROVADO',
       'REPROVADO POR NOTA', 'REPROVADO POR FALTAS',
       'REPROVADO POR NOTA E FALTA'], dtype=object)

In [6]:
# Pegar apenas quem começou a pagar a materia

indesejada = ['DESISTENCIA', 'INDEFERIDO']

matriculas_cet = matriculas_cet[~matriculas_cet.descricao.isin(indesejada)]

In [7]:
# matriculas_cet[matriculas_cet.descricao == 'CANCELADO']

In [8]:
matriculas_cet.shape

(65977, 11)

In [9]:
matriculas_cet.columns

Index(['id_turma', 'discente', 'id_curso', 'unidade', 'nota', 'reposicao',
       'faltas_unidade', 'media_final', 'numero_total_faltas', 'descricao',
       'Unnamed: 10'],
      dtype='object')

In [10]:
novo = matriculas_cet.drop_duplicates(subset=['id_turma', 'discente'])
novo = novo.dropna(subset=['media_final', 'numero_total_faltas'])

In [11]:
novo.shape
novo.head()

,id_turma,discente,id_curso,unidade,nota,reposicao,faltas_unidade,media_final,numero_total_faltas,descricao,Unnamed: 10
293,57576612,86f9f0a3bb404c7499ea9e58128bf75b,10320810.0,1.0,0.0,f,0.0,0.0,20.0,REPROVADO,NaN
296,57576612,4fe88bee3257855047a3682450b8022c,10320810.0,1.0,6.7,f,0.0,5.6,0.0,APROVADO POR NOTA,NaN
299,57576612,bc82dcde9b77887210980114ffd33c42,10320810.0,1.0,5.5,f,0.0,6.1,0.0,APROVADO POR NOTA,NaN
302,57576612,a036073779ed35848a20788046f54bb5,10320810.0,3.0,5.8,f,0.0,5.0,0.0,APROVADO POR NOTA,NaN
305,57576612,bf1b0128073eeb0a9e64bacf7f5c5c81,10320810.0,2.0,3.4,f,0.0,2.2,0.0,REPROVADO,NaN


In [12]:
novo.columns

Index(['id_turma', 'discente', 'id_curso', 'unidade', 'nota', 'reposicao',
       'faltas_unidade', 'media_final', 'numero_total_faltas', 'descricao',
       'Unnamed: 10'],
      dtype='object')

## Extraindo as informações estatísticas

In [13]:
# Calculando qnt_disc
qnt_disc = novo.groupby('id_turma', as_index=False)['media_final'].count() \
                    .rename(columns={'media_final': 'qnt_disc'})

# Calculando media_turma
media_turma = novo.groupby('id_turma', as_index=False)['media_final'].mean() \
                    .rename(columns={'media_final': 'media_turma'})

# Calculando media_faltas
media_faltas = novo.groupby('id_turma', as_index=False)['numero_total_faltas'].mean() \
                    .rename(columns={'numero_total_faltas': 'media_faltas'})

# Calculando qnt_reposicao
qnt_reposicao = novo[novo.reposicao == 't'].groupby('id_turma', as_index=False)['reposicao'].count() \
                    .rename(columns={'reposicao': 'qnt_reposicao'})

# Calculando qnt_aprovados
aprovados = ['APROVADO', 'APROVADO POR NOTA']
qnt_aprovados = novo[novo.descricao.isin(aprovados)].groupby('id_turma', as_index=False)['descricao'].count() \
                    .rename(columns={'descricao': 'qnt_aprovados'})

# Calculando qnt_reprovados
reprovado = ['REPROVADO', 'REPROVADO POR NOTA', 'REPROVADO POR FALTAS', 
             'REPROVADO POR NOTA E FALTA', 'REPROVADO POR MÉDIA E POR FALTAS']
qnt_reprovados = novo[novo.descricao.isin(reprovado)].groupby('id_turma', as_index=False)['descricao'].count() \
                    .rename(columns={'descricao': 'qnt_reprovados'})

# Calculando qnt_trancamento
trancado = ['CANCELADO', 'EXCLUIDA', 'TRANCADO']
qnt_trancamento = novo[novo.descricao.isin(trancado)].groupby('id_turma', as_index=False)['descricao'].count() \
                    .rename(columns={'descricao': 'qnt_trancamento'})

In [14]:
# mesclando os datasets
turmas = pd.merge(left=qnt_disc, right=qnt_aprovados, how='outer')
turmas = pd.merge(left=turmas, right=qnt_reprovados, how='outer')
turmas = pd.merge(left=turmas, right=qnt_trancamento, how='outer')

In [15]:
turmas[turmas.qnt_disc > 100].head()

,id_turma,qnt_disc,qnt_aprovados,qnt_reprovados,qnt_trancamento
0,57573481,112,77.0,32.0,3.0
1,57573482,129,76.0,49.0,4.0
2,57573483,124,78.0,41.0,5.0
3,57573484,114,40.0,65.0,9.0
4,57573485,103,25.0,68.0,10.0


In [16]:
# Realizando o merge com as demais informações
turma_geral = pd.merge(left=turmas, right=media_turma, how='outer')
turma_geral = pd.merge(left=turma_geral, right=media_faltas, how='outer')
turma_geral = pd.merge(left=turma_geral, right=qnt_reposicao, how='outer')

# Preenchendo os valores vazios com 0
turma_geral = turma_geral.fillna(value=0)

In [17]:
turma_geral.to_csv('outputs/turmas_dados_2016_1.csv', sep=';', index=False)